## **1. Tools in LLMs?**

LLMs are powerful at reasoning, pattern recognition, and language generation, but they **cannot directly access external systems** such as:

- the internet  
- databases  
- file systems  
- computational engines  
- custom business logic  

**Tools extend LLM capabilities** by allowing them to call external functions, APIs, or code to perform actions they cannot do alone.

**Examples of capabilities unlocked with tools:**
- Run real code to avoid hallucinated results  
- Query real-time data  
- Perform precise calculations  
- Manipulate files, images, or datasets  
- Act as an orchestrator for multiple services  

---

## **2. When Should You Use Tools?**

Use a tool when:

### **✓ You need accuracy**
LLMs may make up data or math results. A tool (e.g., Python function) ensures correctness.

### **✓ You need external data**
LLMs cannot know live information unless you provide a tool to fetch it.

### **✓ You need actions**
LLMs cannot *do* things like create files, send requests, or run simulations unless tools are provided.

### **✓ You want reproducibility**
A deterministic function eliminates the variability of language generation.

### **✓ You want modularity**
Tools allow complex workflows to be broken into clear, testable components.

---

## **3. How Tools Work (Conceptually)**

An LLM can be given a set of functions (a “toolbox”) where each function is described.  
The LLM decides **if and when** to call a tool based on user requests.

The workflow:

1. User asks a question  
2. LLM interprets the intent  
3. If needed, the LLM calls the tool with structured arguments  
4. The tool executes in Python (or elsewhere)  
5. The LLM uses the results to produce an answer  



In [ ]:
import os
import json
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
aval_api_key=os.getenv("AVALAI_API_KEY")
open_weather_api_key=os.getenv("OPENWEATHER_API_KEY")

In [ ]:
client = OpenAI(    
    api_key=aval_api_key,
    base_url="https://api.avalai.ir/v1"
    )

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini", 
    tools=[{"type": "web_search"}], # comment this line to see the result without using web search
    input="قیمت دلار امروز چنده?",
    temperature=0.1,
)

print(response.output_text)

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini", 
    # tools=[{"type": "web_search"}], # comment this line to see the result without using web search
    input="هوای امروز تهران چطوره؟",
    temperature=0.1,
)

print(response.output_text)

In [ ]:
def get_weather(city: str, unit: str = "metric"):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={open_weather_api_key}&units={unit}"
    resp = requests.get(url)
    data = resp.json()
    return {
        "city": city,
        "temp": data["main"]["temp"],
        "desc": data["weather"][0]["description"]
    }

In [ ]:
# Read about *args and **kwargs in Python 
get_weather(**{'city': 'Tehran', 'unit': 'metric'})

In [ ]:
get_weather(*('Tehran', 'metric'))

In [ ]:
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current weather information for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "Name of the city"},
                "unit": {
                    "type": "string",
                    "enum": ["metric", "imperial"],
                    "description": "Temperature unit"
                }
            },
            "required": ["city"],
        }
    }
]

In [ ]:
user_message = "هوای امروز تهران چطور؟"
input_list = [{"role": "user", "content": user_message}]

response = client.responses.create(
    model="gpt-4o-mini",
    input=input_list,
    tools=tools
)


In [ ]:
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_weather":
            args = json.loads(item.arguments)
            weather = get_weather(**args)
            
            # Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "weather": weather
                })
            })

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions="Respond only with a weather generated by a tool.",
    tools=tools,
    input=input_list,
)

print(response.output_text)